In [ ]:
import os
import os.path as path

import cv2
import numpy as np
from glob import glob

import dlib
import imutils
from imutils import face_utils

In [ ]:
def make_dir(dir_path):
    if not path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path

def video2frame(src, dest, rotate):
    """ Extract raw video frames
    Args:
        src: video file path
        dest: destination folder path
        rotate: rotation type
    """
    print("Extracting frames from {} to {} ...".format(src, dest))
    video = cv2.VideoCapture(glob(path.join(src, "*.avi"))[0])
    flag, frame = video.read()
    count = 0
    while flag:
        if rotate:
            frame = cv2.flip(frame, 1)  # Flip frame
        cv2.imwrite(os.path.join(dest,"{:05}.png".format(count)), frame)
        flag, frame = video.read()
        count += 1
    video.release()
    cv2.destroyAllWindows()
    print("Successfully extract {} frames.".format(count))
    return 0


def dlib_detect(frame_path, crop_path):
    """ Use Dlib to detect eye and save cropped eye image
    Args:
        frame_path: original face frame
        crop_path: cropped eye frame
    """
    # Single frame detection
    def detect(image):
        ih, iw = image.shape[:2]
        image = imutils.resize(image, width=500)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        oh, ow = gray.shape[:2]
        rects = detector(gray, 1)  # A bounding box rectangle produced by a dlib detector
        if len(rects) == 0:
            return False
        else:
            face_bbs = [face_utils.rect_to_bb(rect) for rect in rects]
            # Selects the face with largest area
            index, face_rect = max(enumerate(face_bbs), key=lambda it: it[1][2] * it[1][3])
            shape = predictor(gray, rects[index])  # Shape object containing the 68 (x, y)-coordinates of the facial landmark regions
            shape = np.array(face_utils.shape_to_np(shape))
            right_eye, left_eye = shape[42:48, :], shape[36:42, :]

            def get_bounding_box(eye_shape):
                scale = 1.5
                center = np.mean([np.amin(eye_shape[:, :2], axis=0),
                                  np.amax(eye_shape[:, :2], axis=0)], axis=0)
                ew = (eye_shape[3][0] - eye_shape[0][0]) * scale
                eh = int(0.8 * ew)
                ex = center[0] - ew / 2
                ey = center[1] - eh / 2
                return (int(ex / ow * iw), int(ey / ow * iw), int(ew / ow * iw), int(eh / ow * iw))

            left_bb, right_bb = get_bounding_box(left_eye), get_bounding_box(right_eye)
            
            return left_bb, right_bb

    # Load face landmarks detector
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("D://ASGaze//iris_boundary_detector//data_sources//cvdata//shape_predictor_68_face_landmarks.dat")
    
    make_dir(path.join(crop_path, "left"))
    make_dir(path.join(crop_path, "right"))
    img_fns = glob(path.join(frame_path, "*.png"))
    
    # Length of files
    img_length = len(img_fns)

    count = 0
    info = []

    # Compute rect(x,y,w,h) for per i frame
    for i in range(img_length):
        img = cv2.imread(img_fns[i])
        eyes = detect(img)
        all_valid = all([item != False for item in eyes])
        # Validation check
        if all_valid:
            left_bb = eyes[0]
            right_bb = eyes[1]
            
            left_eye_img = img[left_bb[1]:left_bb[1] + left_bb[3], left_bb[0]:left_bb[0] + left_bb[2]]
            right_eye_img = img[right_bb[1]:right_bb[1] + right_bb[3], right_bb[0]:right_bb[0] + right_bb[2]]

            cv2.imwrite(path.join(crop_path, "left", img_fns[i].split("\\")[-1]), left_eye_img)
            cv2.imwrite(path.join(crop_path, "right", img_fns[i].split("\\")[-1]), right_eye_img)

            info.append(list(left_bb) + list(right_bb))
        else:
            print("No detected eye")
            count += 1
            
    # Save crop information
    root = "/".join(crop_path.split("/")[:-2])
    np.save(path.join(crop_path, "crop.npy"), np.array(info))

    print("Total frames {}. Success rate: {}".format(img_length - count, 1 - count / img_length))

def detection_main(data_path,detection_list,rotate):
    
    for detection in detection_list:
        print(detection)
        viceo_path = path.join(data_path,detection)
        frame_path, crop_path = make_dir(path.join(viceo_path,"frame/")), make_dir(path.join(viceo_path,"crop/"))

        # Extract frames
        video2frame(viceo_path, frame_path, rotate)
        dlib_detect(frame_path, crop_path)
     
    print("Done")